In [1]:
import pandas as pd
import numpy as np
from numpy.core.records import fromarrays
from scipy.io import savemat

In [2]:
choice = 3  #start from 0
devices = ['nfet_01v8_lvt', 'nfet_01v8', 'pfet_01v8_lvt', 'pfet_01v8']

# widths used for characterization and fringe cap parameters (fringe caps are not included in ngspice output)
w = np.array([5, 5, 5, 5])
nfing = np.array([1, 1, 1, 1])
cgdo_w = np.array([2.39289e-10, 2.44907e-10, 2e-11, 5.24893e-11])*1e-6

In [3]:
df_raw = pd.read_csv('./simulation/techsweep_'+devices[choice]+'.txt', sep='\s+')
par_names = df_raw.columns.to_list()
par_prefix = par_names[1].split('[')[0]
print(par_prefix)

@m.xm1.msky130_fd_pr__pfet_01v8


In [4]:
# remove extra headers in file body and unwanted columns
df_raw = df_raw[~df_raw['v-sweep'].str.contains('v-sweep')]
df = df_raw.drop(['v-sweep', 'v-sweep.1'], axis=1)
df = df.apply(pd.to_numeric)

# rename columns for readability
df.columns = df.columns.str.removeprefix(par_prefix+'[')
df.columns = df.columns.str.removesuffix(']')
df

,capbd,capbs,cdd,cgb,cgd,cgg,cgs,css,gds,gm,gmbs,id,l,vth,b,d,g
0,2.813000e-15,2.813000e-15,1.011000e-16,-2.582000e-15,-7.176000e-17,2.620000e-15,3.303000e-17,-1.829000e-17,8.637000e-11,1.862000e-32,6.028000e-33,0.000000,1.500000e-07,0.799,0.0,0.0,0.000
1,2.813000e-15,2.813000e-15,1.195000e-16,-2.560000e-15,-8.482000e-17,2.605000e-15,4.000000e-17,-2.184000e-17,1.684000e-10,0.000000e+00,0.000000e+00,0.000000,1.500000e-07,0.799,0.0,0.0,-0.025
2,2.813000e-15,2.813000e-15,1.411000e-16,-2.538000e-15,-1.002000e-16,2.591000e-15,4.773000e-17,-2.571000e-17,3.261000e-10,0.000000e+00,0.000000e+00,0.000000,1.500000e-07,0.799,0.0,0.0,-0.050
3,2.813000e-15,2.813000e-15,1.665000e-16,-2.517000e-15,-1.181000e-16,2.578000e-15,5.667000e-17,-3.019000e-17,6.252000e-10,0.000000e+00,0.000000e+00,0.000000,1.500000e-07,0.799,0.0,0.0,-0.075
4,2.813000e-15,2.813000e-15,1.962000e-16,-2.495000e-15,-1.392000e-16,2.567000e-15,6.696000e-17,-3.532000e-17,1.184000e-09,0.000000e+00,0.000000e+00,0.000000,1.500000e-07,0.799,0.0,0.0,-0.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255834,1.514000e-15,2.342000e-15,1.100000e-17,-1.456000e-16,-9.979000e-18,8.442000e-14,-8.426000e-14,5.389000e-14,1.094000e-07,6.127000e-05,1.354000e-05,0.000019,3.000000e-06,1.134,0.4,-1.8,-1.700
255835,1.514000e-15,2.342000e-15,1.184000e-17,-4.667000e-17,-1.075000e-17,8.462000e-14,-8.456000e-14,5.402000e-14,1.196000e-07,6.353000e-05,1.409000e-05,0.000020,3.000000e-06,1.134,0.4,-1.8,-1.725
255836,1.514000e-15,2.342000e-15,1.275000e-17,4.934000e-17,-1.157000e-17,8.480000e-14,-8.484000e-14,5.415000e-14,1.304000e-07,6.576000e-05,1.464000e-05,0.000022,3.000000e-06,1.134,0.4,-1.8,-1.750
255837,1.514000e-15,2.342000e-15,1.370000e-17,1.428000e-16,-1.244000e-17,8.497000e-14,-8.510000e-14,5.426000e-14,1.418000e-07,6.796000e-05,1.519000e-05,0.000024,3.000000e-06,1.134,0.4,-1.8,-1.775


In [5]:
# sweep variable vectors
l = (np.unique(abs(df['l']))/1e-6).round(3)
vgs = np.unique(abs(df['g']))
vds = np.unique(abs(df['d']))
vsb = np.unique(abs(df['b']))

In [6]:
# data
# ngspice sweep order is vgs, vds, vsb, l
dims = [len(vgs), len(vds), len(vsb), len(l)]
id = np.reshape(df['id'].values, dims, order='F')
vt = np.reshape(df['vth'].values, dims, order='F')
gm = np.reshape(df['gm'].values, dims, order='F')
gmb = np.reshape(df['gmbs'].values, dims, order='F')
gds = np.reshape(df['gds'].values, dims, order='F')
cfringe = w[choice]*cgdo_w[choice]
cgg = np.reshape(df['cgg'].values, dims, order='F') \
      + 2*cfringe
cgb = -np.reshape(df['cgb'].values, dims, order='F')
cgd = -np.reshape(df['cgd'].values, dims, order='F') \
      + cfringe
cgs = -np.reshape(df['cgs'].values, dims, order='F') \
      + cfringe
cdd = np.reshape(df['cdd'].values, dims, order='F') \
      + np.reshape(df['capbd'].values, dims, order='F') \
      + cfringe
css = np.reshape(df['css'].values, dims, order='F') \
      + np.reshape(df['capbd'].values, dims, order='F') \
      + cfringe

# output order for standard mat file is l, vgs, vds, vsb
id = np.transpose(id, (3, 0, 1, 2))
vt = np.transpose(vt, (3, 0, 1, 2))
gm = np.transpose(gm, (3, 0, 1, 2))
gmb = np.transpose(gmb, (3, 0, 1, 2))
gds = np.transpose(gds, (3, 0, 1, 2))
cgg = np.transpose(cgg, (3, 0, 1, 2))
cgb = np.transpose(cgb, (3, 0, 1, 2))
cgd = np.transpose(cgd, (3, 0, 1, 2))
cgs = np.transpose(cgs, (3, 0, 1, 2))
cdd = np.transpose(cdd, (3, 0, 1, 2))
css = np.transpose(css, (3, 0, 1, 2))

In [7]:
dic = {
  "INFO": "SkyWater, 130nm CMOS, BSIM4",
  "CORNER": "NOM",
  "TEMP": 300.0,
  "VGS": vgs,
  "VDS": vds,
  "VSB": vsb,
  "L": l,
  "W": w[choice],
  "NFING": nfing[choice],
  "ID": id,
  "VT": vt,
  "GM": gm,
  "GMB": gmb,
  "GDS": gds,
  "CGG": cgg,
  "CGB": cgb,
  "CGD": cgd,
  "CGS": cgs,
  "CDD": cdd,
  "CSS": css
}

savemat('./simulation/'+devices[choice]+'.mat', {devices[choice]: dic})